In [100]:
import Constants
from  openai import OpenAI
import os
import numpy as np
import pandas as pd
import time

import gspread
import gspread_dataframe as gd
import gspread_formatting as gf
from gspread_formatting import cellFormat, color, textFormat


## Sample OpenAI Prompt Completion

In [101]:
client = OpenAI(
    api_key = Constants.API_KEY_OPENAI,
)

def get_completion(prompt, model="gpt-4o-mini", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response

## Test Google Sheet API

In [102]:
# No need to provide path for service account.  Json file is in the default directory for gspread at %APPDATA%/gspread/service_account.json
sa = gspread.service_account()
sh = sa.open("New Chinese Words")

sheet_name = "Words"
wks2 = sh.worksheet(sheet_name)
print(f'Sheet {sheet_name} Num Rows: {wks2.row_count}')
print(f'Sheet {sheet_name} Num Columns: {wks2.col_count}')


Sheet Words Num Rows: 99
Sheet Words Num Columns: 8


In [103]:
current_data = pd.DataFrame(wks2.get_all_values())
current_data.columns = current_data.iloc[0]
current_data = current_data.iloc[1:]
current_data.head()

,Word,Pinyin,Type,Meaning,Sentence,Sentence Pinyin,Sentence Meaning
1,大概,da4 gai4,Adjective,Approximately,他大概已经出发了,Tā dàgài yǐjīng chūfā le.,He probably already left
2,被,bei4,Particle,Particle word to turn into passive voice. (.i...,甜甜圈被吃掉了,tian2 tian2 quan1 bei4 chi1 diao4 le,The donut was eaten
3,往前,wang3 qian2,Adjective,Go forward,往前走,wang3 qian2 zou3,
4,觉得,jue2 de,Verb,Think,你觉得她怎么样,ni3 jue2 de ta1 zen3 me yang,What do you think about her
5,已经,yi3 jing1,Adverb,Already,我已经吃过晚饭了,Wǒ yǐjīng chī guò wǎnfàn le.,I have already had dinner


## Chinese Language Translation

In [104]:
from main.translation import *

In [105]:
temp = 0.7
overwrite_mode = False

dict_path = "ChineseWords/ChineseWordList.csv"
dict_sheet_name = "AutoWordList"
gsheet_name = "New Chinese Words"


df = load_dict(gsheet_mode=True, gsheet_name=gsheet_name, worksheet_name=dict_sheet_name)
cat = df['Word Category'].drop_duplicates().sort_values().to_list()

In [106]:
sample_response_translation = get_completion(prompt=get_prompt_for_chinese_translation("诺言, 意思, 听, 思绪, 充满, 疑惑, 寻找, 表现, 很大方"), temperature=temp)
content = sample_response_translation.choices[0].message.content
print(content)

| Word   | Pinyin      | Pinyin Simplified | Type    | Meaning         | Sentence                             | Sentence Pinyin                                  | Sentence Meaning                                         | Word Category |
|--------|-------------|--------------------|---------|------------------|--------------------------------------|---------------------------------------------------|---------------------------------------------------------|---------------|
| 件杯   | jiàn bēi    | jian4 bei1         | Noun    | Cup              | 我喝水的时候喜欢用这件杯。             | Wǒ hē shuǐ de shíhòu xǐhuān yòng zhè jiàn bēi.   | I like to use this cup when I drink water.             | Object        |
| 瓶    | píng        | ping2              | Noun    | Bottle           | 她用一个瓶子装水。                   | Tā yòng yīgè píngzi zhuāng shuǐ.                  | She used a bottle to hold water.                        | Object        |
| 双    | shuāng      | shuang1            | Noun    | Pair          

In [107]:
newwords_df = parse_translation_response(content)
new_words = newwords_df['Word'].drop_duplicates().values
newwords_df

,Word,Pinyin,Pinyin Simplified,Type,Meaning,Sentence,Sentence Pinyin,Sentence Meaning,Word Category,Added Date
1,件杯,jiàn bēi,jian4 bei1,Noun,Cup,我喝水的时候喜欢用这件杯。,Wǒ hē shuǐ de shíhòu xǐhuān yòng zhè jiàn bēi.,I like to use this cup when I drink water.,Object,2024-10-13
2,瓶,píng,ping2,Noun,Bottle,她用一个瓶子装水。,Tā yòng yīgè píngzi zhuāng shuǐ.,She used a bottle to hold water.,Object,2024-10-13
3,双,shuāng,shuang1,Noun,Pair,我买了一双鞋子。,Wǒ mǎile yī shuāng xiézi.,I bought a pair of shoes.,Object,2024-10-13
4,辆,liàng,liang4,Noun,Vehicle,他有一辆车。,Tā yǒu yī liàng chē.,He has a vehicle.,Object,2024-10-13
5,片,piàn,pian4,Noun,Piece,我切了一片面包。,Wǒ qiēle yī piàn miànbāo.,I cut a piece of bread.,Object,2024-10-13
6,口,kǒu,kou3,Noun,Mouth,他的口很大。,Tā de kǒu hěn dà.,His mouth is very big.,Object,2024-10-13
7,头,tóu,tou2,Noun,Head,这头牛很大。,Zhè tóu niú hěn dà.,This head of cattle is very large.,Object,2024-10-13


In [108]:
save_new_words_to_dict(
    newwords_df = newwords_df,
    gsheet_mode= True,
    overwrite_mode = True,
    gsheet_name = gsheet_name,
    worksheet_name = dict_sheet_name)

Overwrite mode enabled.  Replacing 0 words and 7 new words added.


## Mandarin Word Quiz

In [274]:
from main.quiz import *

In [154]:
word_dict = load_dict(gsheet_mode=True, gsheet_name=gsheet_name, worksheet_name=dict_sheet_name)
word_dict.head()

,Word Id,Word,Pinyin,Pinyin Simplified,Type,Meaning,Sentence,Sentence Pinyin,Sentence Meaning,Word Category,Added Date,Num_Quiz,Num_Correct,Num_Wrong
1,1,帮助,bang1 zhu4,bang1 zhu4,Noun/Verb,Help/Assistance,我可以帮助你学习中文,Wǒ kěyǐ bāngzhù nǐ xuéxí Zhōngwén.,I can help you study Chinese.,Support,2024-09-21,0,0,0
2,2,包裹,bao1 guo3,bao1 guo3,Noun,Package,他昨天收到了一个包裹。,Wǒ zuótiān shōudào le bāoguǒ.,I received the package yesterday.,Object,2024-09-21,0,0,0
3,3,比较,bi3 jiao4,bi3 jiao4,Adjuster,Comparatively/More (More about comparing the t...,这家餐厅的菜比较好,Zhè jiā cāntīng de cài bǐjiào hǎo.,The dishes at this restaurant are relatively b...,Degree,2024-09-21,0,0,0
4,4,不错,bu2 cuo4,bu2 cuo4,Adjective,Good (More positive than 还好),这个电影不错，我们可以一起去看。,Zhè gè diànyǐng bùcuò wǒmen kěyǐ yìqǐ qù kàn,This movie is pretty good; we can go watch it ...,Opinion,2024-09-21,0,0,0
5,5,不过,bu2 guo4,bu2 guo4,Grammar,But (Similar to dan4 shi4 but less formal),我喜欢这家餐厅，不过价格有点贵。,Wǒ xǐhuān zhè jiā cāntīng búguò jiàgé yǒudiǎn ...,I like this restaurant but the price is a bit ...,Grammar,2024-09-21,0,0,0


In [271]:
quiz_df_show, answer_key = generate_pinyin_and_meaning_quiz(
    word_dict, 
    id_column='Word Id', 
    random_params={
        'num_words': 10, 
        'date_filter': '2024-10-13', 
        #'category_filter': 'Measure Word'
        }
    )

quiz_df_no_id = quiz_df_show.drop(columns=['Word Id'])
quiz_df_no_id

,Word,Sentence
0,情绪,她的情绪一直很低落。
1,头,这头牛很大。
2,相聚,朋友们都在这里相聚。
3,仔细,请仔细阅读说明书。
4,脚踝,我扭伤了脚踝，走路很困难。
5,设计,他在设计一款新手机。
6,恰好,他恰好赶上了最后一班车。
7,颗,我吃了一颗苹果。
8,片,我切了一片面包。
9,呼吸,我在瑜伽课上学会了如何正确地呼吸。


In [272]:
pinyin_answer = ['something', 'tou2', 'xiang1 ju4', 'something', 'something something', 'she4 ji4', 'qia4 hao3', 'ke1', 'pian4', 'hu1 xi1']
meaning_answer = ['dunno', 'Measure word for large animal', 'to meet', 'weird', 'meh', 'to design', 'just right', 'Measure Word for Round Things', 'Measure word for thinly sliced object', 'breath']

In [273]:
quiz_result = evaluate_pinyin_and_meaning_quiz(
    answer_key_df = answer_key, 
    quiz_df = quiz_df_show, 
    pinyin_answer = pinyin_answer, 
    meaning_answer = meaning_answer
)

quiz_result

,Word,Sentence,Pinyin Answer,Pinyin Correct,Pinyin Correction,Meaning,Meaning Correct,Meaning Correction
0,情绪,她的情绪一直很低落。,something,no,qíng xù,dunno,no,"Incorrect meaning; it means ""emotion""."
1,头,这头牛很大。,tou2,yes,,Measure word for large animal,yes,
2,相聚,朋友们都在这里相聚。,xiang1ju4,yes,,to meet,yes,
3,仔细,请仔细阅读说明书。,something,no,zǐxì,weird,no,"Incorrect meaning; it means ""carefully""."
4,脚踝,我扭伤了脚踝，走路很困难。,somethingsomething,no,jiǎo huái,meh,no,"Incorrect meaning; it means ""ankle""."
5,设计,他在设计一款新手机。,she4ji4,yes,,to design,yes,
6,恰好,他恰好赶上了最后一班车。,qia4hao3,yes,,just right,yes,
7,颗,我吃了一颗苹果。,ke1,yes,,Measure Word for Round Things,yes,
8,片,我切了一片面包。,pian4,yes,,Measure word for thinly sliced object,yes,
9,呼吸,我在瑜伽课上学会了如何正确地呼吸。,hu1xi1,yes,,breath,yes,


In [278]:
quiz_result_export = quiz_result.copy()
quiz_result_export['Right Score'] = np.where((quiz_result_export['Meaning Correct']=="yes")|(quiz_result_export['Pinyin Correct']=="yes"), 1, 0)
quiz_result_export['Wrong Score'] = np.where(quiz_result_export['Right Score']==0, 1, 0)
quiz_result_export

,Word,Sentence,Pinyin Answer,Pinyin Correct,Pinyin Correction,Meaning,Meaning Correct,Meaning Correction,Right Score,Wrong Score
0,情绪,她的情绪一直很低落。,something,no,qíng xù,dunno,no,"Incorrect meaning; it means ""emotion"".",0,1
1,头,这头牛很大。,tou2,yes,,Measure word for large animal,yes,,1,0
2,相聚,朋友们都在这里相聚。,xiang1ju4,yes,,to meet,yes,,1,0
3,仔细,请仔细阅读说明书。,something,no,zǐxì,weird,no,"Incorrect meaning; it means ""carefully"".",0,1
4,脚踝,我扭伤了脚踝，走路很困难。,somethingsomething,no,jiǎo huái,meh,no,"Incorrect meaning; it means ""ankle"".",0,1
5,设计,他在设计一款新手机。,she4ji4,yes,,to design,yes,,1,0
6,恰好,他恰好赶上了最后一班车。,qia4hao3,yes,,just right,yes,,1,0
7,颗,我吃了一颗苹果。,ke1,yes,,Measure Word for Round Things,yes,,1,0
8,片,我切了一片面包。,pian4,yes,,Measure word for thinly sliced object,yes,,1,0
9,呼吸,我在瑜伽课上学会了如何正确地呼吸。,hu1xi1,yes,,breath,yes,,1,0


In [276]:
word_dict.head()

,Word Id,Word,Pinyin,Pinyin Simplified,Type,Meaning,Sentence,Sentence Pinyin,Sentence Meaning,Word Category,Added Date,Num_Quiz,Num_Correct,Num_Wrong
1,1,帮助,bang1 zhu4,bang1 zhu4,Noun/Verb,Help/Assistance,我可以帮助你学习中文,Wǒ kěyǐ bāngzhù nǐ xuéxí Zhōngwén.,I can help you study Chinese.,Support,2024-09-21,0,0,0
2,2,包裹,bao1 guo3,bao1 guo3,Noun,Package,他昨天收到了一个包裹。,Wǒ zuótiān shōudào le bāoguǒ.,I received the package yesterday.,Object,2024-09-21,0,0,0
3,3,比较,bi3 jiao4,bi3 jiao4,Adjuster,Comparatively/More (More about comparing the t...,这家餐厅的菜比较好,Zhè jiā cāntīng de cài bǐjiào hǎo.,The dishes at this restaurant are relatively b...,Degree,2024-09-21,0,0,0
4,4,不错,bu2 cuo4,bu2 cuo4,Adjective,Good (More positive than 还好),这个电影不错，我们可以一起去看。,Zhè gè diànyǐng bùcuò wǒmen kěyǐ yìqǐ qù kàn,This movie is pretty good; we can go watch it ...,Opinion,2024-09-21,0,0,0
5,5,不过,bu2 guo4,bu2 guo4,Grammar,But (Similar to dan4 shi4 but less formal),我喜欢这家餐厅，不过价格有点贵。,Wǒ xǐhuān zhè jiā cāntīng búguò jiàgé yǒudiǎn ...,I like this restaurant but the price is a bit ...,Grammar,2024-09-21,0,0,0


In [ ]:
answer_key

In [ ]:
word_dict

## Prompt Mentoring

In [69]:
client = OpenAI(api_key="<Insert yours>")

equality_statement = "Interprete this 'Edge_PB_Share_Avg > 0.82', 'EnterpriseMobilityCoreE3Rev > 1907.40', '5921.00 < AADPAllUp_MAU <= 15701.50',"

def interpret_equality_statement(equality_statement):
    # Initialize variables for tracking attempts
    attempt_count = 0
    max_attempts = 5
    highest_score = 0
    feedback_dict = {}
 
    while attempt_count < max_attempts:
        # MLS Simulation: Generate interpretation
        interpretation = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": """You are a Machine Learning Scientist who converts equality statements into proper English.
                        An equality statement has the pattern "<measure> <equality> <value>" or "<value> <equality> <measure> <equality> <value>".

                        <measure> has a mapping to proper English:
                        Edge_PB_Share_Avg : Edge Primary Browser Share
                        SPO_MAU : SharePoint Online MAU
                        EnterpriseMobilityCoreE3Rev : EMS E3 Revenue
                        AADPAllUp_MAU : Entra ID MAU

                        if <value> is less than 1, it is a percentage.
                        if <value> is greater than 999, it must have comma(s).
                        if <value> is greater than 1, it must be rounded to the nearest whole number.
                        if <measure> has MAU, round <value> to the nearest hundred.

                        You always start an English sentence with the phrase "We recommend this product for this tenant because..."

                    """
                },
                {
                    "role": "user",
                    "content": equality_statement
                }
            ]
        )
        generated_interpretation = interpretation.choices[0].message.content

        # Mentor Simulation: Generate mentor feedback
        mentor_feedback = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {
                    "role": "system",
                    "content": """You are a senior leader in an organization who evaluates the output of the Machine Learning Scientist. You will give them a score of their work from 1 to 10, and provide reasons and comments.

                        Evaluation criteria:
                        1. The equality statement should have the pattern "<measure> <equality> <value>" or "<value> <equality> <measure> <equality> <value>".
                        2. The statement needs to be mapped to proper English:
                            - Edge_PB_Share_Avg: Edge Primary Browser Share
                            - SPO_MAU: SharePoint Online MAU
                            - EnterpriseMobilityCoreE3Rev: EMS E3 Revenue
                            - AADPAllUp_MAU: Entra ID MAU
                        3. If <value> is less than 1, it's a percentage.
                        4. If <value> is greater than 999, it should have commas.
                        5. If <value> is greater than 1, round to the nearest whole number.
                        6. If the <measure> has MAU, round the <value> to the nearest hundred.
                        7. The English sentence should always start with the phrase "We recommend this product for this tenant because...".
                        8. The response should be in professional English.
                        9. Your mentee should not say the result is rounded or anything related to how they finish the process in the response, they should stick with the precise interpretation

                        # Good Examples:
                        Sharepoint Online usage is above 80%
                        Azure Revenue is more than $1000
                        the Entra ID MAU is between 5,900 and 15,700

                        # Bad Examples:
                        the Entrta ID MAU is between 5921 and 15701, rounded to the nearest hundred

                     Sample output: Score: 8.5; Comments: This is good. You missed the SPO_MAU : SharePoint Online MAU converter
                    This is just a sample output, you don't have to follow 100%, but think and give your feedback, if there is no mistake, you can give a 10 as well!
                    But you need to strictly follow the sample output format with Score:<your score, one number or float>; Comments:< Your comments>  
                    """
                },
                {
                    "role": "user",
                    "content": f"Evaluate this work and provide a score and comment.\nOriginal equality statement: {equality_statement}\nWork: {generated_interpretation}"
                }
            ]
        )

        # Parse mentor's feedback
        mentor_response = mentor_feedback.choices[0].message.content
        score = float(mentor_response.split('Score: ')[1].split(';')[0])
        comment = mentor_response.split('Comments: ')[1]
 
        # Check if the score is acceptable
        if score >= 8.5:
            return f"Interpretation Approved: {generated_interpretation}"
 
        # If score is below 8.5, store the feedback
        feedback_dict[attempt_count] = {'score': score, 'comment': comment}
        highest_score = max(highest_score, score)

        # Output feedback and ask user to revise with mentor's comment attached
        print(f"Attempt {attempt_count + 1}: Score: {score}, Feedback: {comment}")
        equality_statement = input(f"Please revise your equality statement based on the mentor's feedback:\n{comment}\nYour revised statement: ")

        attempt_count += 1
 
    # If the loop ends without a passing score, output the highest score
    return f"Max attempts reached. Highest score: {highest_score}, Feedback from last attempt: {feedback_dict[attempt_count-1]['comment']}"


# Example usage
result = interpret_equality_statement("Interprete this 'Edge_PB_Share_Avg > 0.82', 'EnterpriseMobilityCoreE3Rev > 1907.40', '5921.00 < AADPAllUp_MAU <= 15701.50' to english")
print(result)

             

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: <Insert **urs>. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}